# Movie Chain

### Put Your OPENROUTER_API_KEY here

In [ ]:
import os
os.environ["OPENROUTER_API_KEY"] = "paste_your_api_key_here"

In [ ]:
!pip3 install langchain langchain_openai

In [ ]:
import os
from langchain_openai import ChatOpenAI

try:
    model_name
except NameError:
    model_name="openai/gpt-4o-mini"

print("Model Name:", model_name)
print("Provider:", "OpenRouter AI")

llm=ChatOpenAI(model_name=model_name,
               openai_api_key=os.environ.get("OPENROUTER_API_KEY"),
               openai_api_base="https://openrouter.ai/api/v1")

In [ ]:
import textwrap
from langchain_core.prompts import ChatPromptTemplate

from IPython.display import display, clear_output, Markdown
from ipywidgets import widgets, Layout

conversation_output = widgets.Output()
messages = []

def run_chatbot(system_prompt, initial_message):
    global messages 
    messages = [ {'role':'system', 'content': system_prompt} ]
    conversation_output.clear_output()

    messages.append({'role': 'assistant', 'content': initial_message})

    text_input = widgets.Text(
        placeholder='Type your message here...',
        layout=widgets.Layout(width='50%')
    )
    submit_button = widgets.Button(description="Send")

    input_box = widgets.HBox([text_input, submit_button])
    display(conversation_output, input_box)

    def on_submit_click(b):
        message = text_input.value
        text_input.value = ''  # Clear the input field

        with conversation_output:
            display(Markdown(f"**User**: {message}"))
            messages.append({'role': 'user', 'content': message})
            response = get_completion_messages(messages)
            display(Markdown(f"**AI**: {response}"))
            messages.append({'role': 'assistant', 'content': response})

    submit_button.on_click(on_submit_click)

    # Display initial AI message
    with conversation_output:
        display(Markdown(f"**AI**: {initial_message}"))

def wrap_text(text, max_width=80):
    """
    Wraps the text to the specified max_width, preserving line breaks and formatting.
    """
    text = text.lstrip()
    lines = text.splitlines()  # Split the text into lines
    wrapped_lines = []
    for line in lines:
        if line.strip():  # Skip empty lines
            wrapped_line = textwrap.fill(line, max_width, initial_indent='', subsequent_indent='')
            wrapped_lines.extend(wrapped_line.splitlines())  # Preserve line breaks
        else:
            wrapped_lines.append('')  # Keep empty lines
    return '\n'.join(wrapped_lines)

def print_messages():
    for message in messages:
        role = message['role']
        content = message['content']
        
        if role == 'system':
            print("System:")
            print("-" * 40)
            print(content)
        elif role == 'user':
            print("User: ", end="")
            print(wrap_text(content))
        elif role == 'assistant':
            print("Assistant: ", end="")
            print(wrap_text(content))
        print()  # Add an extra newline for spacing

def print_prompt_and_response(prompt, response):
    print("Prompt: ")
    print(wrap_text(prompt))
    print("")
    print("Response: ")
    print(response)

def print_messages_and_response(messages, response):
    prompt = ChatPromptTemplate(messages=messages)
    print_prompt_and_response(prompt.format(), response)

def get_completion(prompt, temperature=0.0):
    response = llm.invoke(prompt, temperature=temperature)
    wrapped_response = wrap_text(response.content)
    return wrapped_response

def get_completion_messages(messages, temperature=0.0):
    response=llm.invoke(messages, temperature=temperature)
    wrapped_response = wrap_text(response.content)
    return wrapped_response

## Movie Chain

We will analyze an extended movie plot summary, identify the genre(s) of
the film, generate a tagline, and then create a short synopsis
incorporating the tagline and genres. The process involves the following
steps:

1.  **Initial Movie Plot**: The process starts with an extended movie
    plot summary, which is provided as a string variable.

2.  **Identify the Film Genre(s)**: The language model will identify the
    primary genre(s) for the film based on the movie plot summary.

3.  **Generate a Movie Tagline**: Based on the initial movie plot, a
    catchy tagline for the movie is generated by prompting a language
    model.

4.  **Generate a Title**: Based on the initial movie plot and the movie
    tagline generate a possible title.

5.  **Generate a Short Synopsis**: Using the initial movie plot, the
    generated tagline and title, and the identified genre(s), generate a
    short synopsis for the movie.

![Movie Chain
Flow](https://calvinw.github.io/Ma322Notebooks/prompt-chain-graph.png)

### Initial Movie Plot

Let’s start with the extended movie plot summary:

In [ ]:
movie_plot ="""
In a post-apocalyptic world ravaged by a devastating cataclysm, a young woman
named Tara sets out on a perilous journey to find a rumored sanctuary known as
The Oasis – a legend that promises safety, resources, and a chance at
rebuilding civilization.

Tara's path is fraught with danger as she navigates the treacherous wastelands,
where lawlessness and chaos reign supreme. Along the way, she encounters a
ragtag group of survivors who have banded together for protection, forming an
unlikely alliance born out of necessity.

Despite initial mistrust and conflicting agendas, Tara and the survivors
realize that their only hope of survival lies in cooperation. They must
traverse a hostile landscape, facing off against ruthless marauders who will
stop at nothing to claim what little resources remain, as well as mutated
creatures twisted by the cataclysm's aftermath.

As they uncover clues and follow a trail of breadcrumbs left by those who came
before, the group is forced to confront not only the harsh realities of their
new world but also their own inner demons and personal conflicts. They must
make difficult choices that will test their morality, their loyalty, and their
resolve, ultimately determining their fate and the fate of those who may follow
in their footsteps.

In this harrowing journey, the line between survival and sacrifice blurs, and
the true cost of reaching The Oasis may be higher than any of them could have
imagined.
"""

### Identify the Film Genre(s)

Identify the primary genre(s) for the film based on the movie plot
summary.

In [ ]:
template ="""
Based on the following movie plot summary, identify the primary genre(s) for
the film. The plot is enclosed in <plot> </plot> tags:

<plot>
{plot}
</plot>

Provide your answer in a comma-separated list:
"""

prompt = template.format(plot=movie_plot)
genre_response = get_completion(prompt)
print(genre_response)

### Generate a Movie Tagline

Generate a catchy tagline for the movie based on the initial movie plot.

In [ ]:
template ="""
Based on the following movie plot, generate a catchy tagline for the movie.
The plot is enclosed in <plot> </plot> tags:

<plot>
{plot}
</plot>
"""

prompt = template.format(plot=movie_plot)
tagline_response = get_completion(prompt)
print(tagline_response)

### Create a Title

Based on the tagline and the plot summary, generate a title for the
film.

In [ ]:
template ="""
Based on the following movie plot summary and tagline generate a title for the
movie. The plot is enclosed in <plot> </plot> tags and the tagline is enclosed
in <tagline> </tagline> tags

<plot>
{plot}
</plot>

<tagline>
{tagline}
</tagline>
"""

prompt = template.format(plot=movie_plot,
                         tagline=tagline_response)
title_response = get_completion(prompt)
print(title_response)

### Generate a Short Synopsis

Generate a short synopsis for the movie, incorporating the tagline and
the identified genre(s).

In [ ]:
template = """
Based on the following movie plot, tagline, title, and genres, generate a short
synopsis for the movie of 50 words or less. The plot is contained in <plot>
tags, the tagline is contained in <tagline> tags, the genres are contained
<genres> tags, the title is contained in <title> tags. 

<plot>
{plot}
</plot>

<genres> 
{genres}
</genres> 

<tagline>
{tagline}
</tagline>

<title>
{title}
</title>
"""

prompt = template.format(plot=movie_plot,
                        tagline=tagline_response,
                        genres=genre_response,
                        title=title_response)
synopsis_response = get_completion(prompt)
print(synopsis_response)

### Final Report for Movie Chain

In [ ]:
report_template = """
### Initial Movie Plot
{plot}

### Identified Genre(s)
{genre}

### Movie Tagline
{tagline}

### Movie Title
{title}

### Short Movie Synopsis
{synopsis}
"""

report = report_template.format(plot = movie_plot,
                         genre = genre_response,
                         tagline = tagline_response,
                         title = title_response,
                         synopsis = synopsis_response)
print(report)